In [2]:
import pandas as pd
import re
from utils import *
import datetime as dt
import glob

In [2]:
input_file = 'Data/ukb45839.tab'

In [3]:
column_names = pd.read_csv(input_file, sep="\t", nrows=0).columns

In [ ]:
column_names[15028]

In [14]:
columns_to_read

['f.eid',
 'f.41202.0.0',
 'f.41202.0.1',
 'f.41202.0.2',
 'f.41202.0.3',
 'f.41202.0.4',
 'f.41202.0.5',
 'f.41202.0.6',
 'f.41202.0.7',
 'f.41202.0.8',
 'f.41202.0.9',
 'f.41202.0.10',
 'f.41202.0.11',
 'f.41202.0.12',
 'f.41202.0.13',
 'f.41202.0.14',
 'f.41202.0.15',
 'f.41202.0.16',
 'f.41202.0.17',
 'f.41202.0.18',
 'f.41202.0.19',
 'f.41202.0.20',
 'f.41202.0.21',
 'f.41202.0.22',
 'f.41202.0.23',
 'f.41202.0.24',
 'f.41202.0.25',
 'f.41202.0.26',
 'f.41202.0.27',
 'f.41202.0.28',
 'f.41202.0.29',
 'f.41202.0.30',
 'f.41202.0.31',
 'f.41202.0.32',
 'f.41202.0.33',
 'f.41202.0.34',
 'f.41202.0.35',
 'f.41202.0.36',
 'f.41202.0.37',
 'f.41202.0.38',
 'f.41202.0.39',
 'f.41202.0.40',
 'f.41202.0.41',
 'f.41202.0.42',
 'f.41202.0.43',
 'f.41202.0.44',
 'f.41202.0.45',
 'f.41202.0.46',
 'f.41202.0.47',
 'f.41202.0.48',
 'f.41202.0.49',
 'f.41202.0.50',
 'f.41202.0.51',
 'f.41202.0.52',
 'f.41202.0.53',
 'f.41202.0.54',
 'f.41202.0.55',
 'f.41202.0.56',
 'f.41202.0.57',
 'f.41202.0.58

In [22]:
_, _, _, _, _, set3eids, _ = get_data({'dset':'cmb', 'target':'mort'})

In [7]:
# Read the data
case_cols = [f"{column_names[i]}" for i in range(14962, 15028)]
date_cols = [f"{column_names[i]}" for i in range(15717, 15783)]
columns_to_read = ["f.eid"] + case_cols + date_cols
icd10_all = pd.read_csv(input_file, sep="\t", usecols=columns_to_read, dtype=str)

In [8]:
set3eids

array([5527725, 5527748, 5527912, ..., 4775604, 2041221, 2819283])

In [17]:
icd10_all['f.eid'].astype('int')

0         1000014
1         1000023
2         1000030
3         1000041
4         1000059
           ...   
502481    6024824
502482    6024838
502483    6024846
502484    6024852
502485    6024869
Name: f.eid, Length: 502486, dtype: int64

In [20]:
icd10 = icd10_all[icd10_all['f.eid'].astype('int').isin(set3eids)]

In [23]:
icd10.shape

(4071, 133)

In [89]:
# Define disease categories and corresponding ICD-10 codes as given by Tigist
diseases = {
    "IschemicHeartDisease": ["I20", "I21", "I22", "I23", "I24", "I25"],
    "Stroke": ["I60", "I61", "I62", "I63", "I64"],
    "HeartFailure": ["I50"],
    "PeripheralArterialDisease": ["I70", "I73"],
    "AtrialFibrillation": ["I48"],
}

def get_earliest_time (row):
    col_idx = row['col_idx']
    lowest_date = dt.date.today()
    
    date_str = "f.41262.0."

    for idx in col_idx:
        date = pd.to_datetime(row[date_str + str(idx)])
        if date.date() < lowest_date:
            date = lowest_date
            lowest_date_col = date_str + str(idx)
    return row[lowest_date_col]

# Define a function to process each disease
def process_disease(disease_name, icd_codes, df):
    # Combine ICD-10 codes into a regex pattern
    icd_pattern = "|".join(icd_codes)
    
    # Identify rows where any of the ICD columns match the codes
    icd_columns = [col for col in df.columns if col in case_cols]
    df_icd = df[df[icd_columns].apply(lambda row: row.str.contains(icd_pattern, na=False).any(), axis=1)]
    df_icd[disease_name] = 1

    #Save all columns that match icd pattern
    df_icdtrue = df_icd[icd_columns].apply(lambda row: row.str.contains(icd_pattern, na=False), axis=1)
    
    #Match date columns row wise 
    df_icd['col_idx'] = df_icdtrue.apply(lambda row: np.where(row.values==True)[0], axis=1)
    df_icd[f'{disease_name}_date'] = df_icd.apply(lambda row: get_earliest_time(row), axis =1)
    df_icd = df_icd[['f.eid', disease_name, f'{disease_name}_date']]

    # Save results for this disease
    output_file = f"Data/merge_{disease_name}.date.csv"
    df_icd.to_csv(output_file, index=False)
    print(f"Processed {disease_name}: {len(df_icd)} rows saved to {output_file}")

# Process each disease
for disease, codes in diseases.items():
    process_disease(disease, codes, icd10)

print("Processing complete. Files saved in Data")


/tmp/ipykernel_2325439/3570994592.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_icd[disease_name] = 1
/tmp/ipykernel_2325439/3570994592.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_icd['col_idx'] = df_icdtrue.apply(lambda row: np.where(row.values==True)[0], axis=1)
/tmp/ipykernel_2325439/3570994592.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentatio

Processed IschemicHeartDisease: 367 rows saved to merge_IschemicHeartDisease.date.csv


/tmp/ipykernel_2325439/3570994592.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_icd[disease_name] = 1
/tmp/ipykernel_2325439/3570994592.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_icd['col_idx'] = df_icdtrue.apply(lambda row: np.where(row.values==True)[0], axis=1)
/tmp/ipykernel_2325439/3570994592.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentatio

Processed Stroke: 101 rows saved to merge_Stroke.date.csv


/tmp/ipykernel_2325439/3570994592.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_icd[disease_name] = 1
/tmp/ipykernel_2325439/3570994592.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_icd['col_idx'] = df_icdtrue.apply(lambda row: np.where(row.values==True)[0], axis=1)
/tmp/ipykernel_2325439/3570994592.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentatio

Processed HeartFailure: 57 rows saved to merge_HeartFailure.date.csv


/tmp/ipykernel_2325439/3570994592.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_icd[disease_name] = 1
/tmp/ipykernel_2325439/3570994592.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_icd['col_idx'] = df_icdtrue.apply(lambda row: np.where(row.values==True)[0], axis=1)
/tmp/ipykernel_2325439/3570994592.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentatio

Processed PeripheralArterialDisease: 31 rows saved to merge_PeripheralArterialDisease.date.csv
Processed AtrialFibrillation: 133 rows saved to merge_AtrialFibrillation.date.csv
Processing complete. Files saved in Data


/tmp/ipykernel_2325439/3570994592.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_icd[disease_name] = 1
/tmp/ipykernel_2325439/3570994592.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_icd['col_idx'] = df_icdtrue.apply(lambda row: np.where(row.values==True)[0], axis=1)
/tmp/ipykernel_2325439/3570994592.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentatio

In [71]:
file_pattern = "Data/merge_*.date.csv"

cvd_dataframes = [pd.read_csv(file) for file in glob.glob(file_pattern)]

# Merge all dataframes on the 'f.eid' column
cvd_merged_df = cvd_dataframes[0]
for df in cvd_dataframes[1:]:
    cvd_merged_df = pd.merge(cvd_merged_df, df, on="f.eid", how="outer")
cvd_merged_df['eid'] = cvd_merged_df['f.eid']
cvd_merged_df['CVD_inc'] = 1
cvd_merged_df = cvd_merged_df.apply(lambda col: pd.to_datetime(col, errors="coerce") if col.name.endswith("date") else col)
cvd_merged_df['CVD_date'] = cvd_merged_df[[col for col in cvd_merged_df.columns if col.endswith("date")]].min(axis =1)
cvd_merged_df['CVD_date'] 
max_date = cvd_merged_df[[col for col in cvd_merged_df.columns if col.endswith("date")]].max().max()
cvd_merged_df = cvd_merged_df[['eid', 'CVD_inc', 'CVD_date']]
#cvd_merged_df

In [72]:
controls = pd.DataFrame(set3eids, columns = ['eid'])
controls = controls[~controls['eid'].isin(cvd_merged_df['eid'])]
controls['CVD_date'] = max_date + pd.Timedelta(days=1)
controls['CVD_inc'] = 0

cvd_set = cvd_merged_df.merge(controls, how = 'outer')
#cvd_set.to_csv('Data/FollowUpCVD.csv', index = False)

In [133]:
basicinfo = pd.read_csv("Data/basicinfo_instance_0.csv")
basicinfo = basicinfo[basicinfo['eid'].isin(cvd_set['eid'])]
cvd_set_time = cvd_set.merge(basicinfo[['eid', 'date_center.0.0', 'age_center.0.0']], on = 'eid', how = 'inner')
cvd_set_time['followup'] = cvd_set_time['CVD_date'] - pd.to_datetime(cvd_set_time['date_center.0.0'])
cvd_set_time['CVD_prev'] = (cvd_set_time['followup'] < pd.Timedelta(0)).astype(int)
cvd_set_time['CVD_age'] = np.where(
    cvd_set_time['CVD_prev'] == 0, 
    (cvd_set_time['followup'].dt.days / 365.25) + cvd_set_time['age_center.0.0'], 
    pd.NA
)
cvd_set_time['CVD_inc'] = np.where(
    cvd_set_time['CVD_prev'] == 1, 
    pd.NA, cvd_set_time['CVD_inc']
)
cvd_set_time['CVD_prev'].sum()
cvd_set_time_save = cvd_set_time[['eid', 'CVD_prev','CVD_inc', 'CVD_age']]
cvd_set_time_save.to_csv('Data/FollowUpCVD.csv', index = False)

In [81]:
# ukb_outcomes = pd.read_csv('Data/ukb675807.csv', nrows=0).columns
# list(ukb_outcomes)

['eid',
 '3-0.0',
 '3-1.0',
 '3-2.0',
 '3-3.0',
 '4-0.0',
 '4-1.0',
 '4-2.0',
 '4-3.0',
 '5-0.0',
 '5-1.0',
 '5-2.0',
 '5-3.0',
 '6-0.0',
 '6-1.0',
 '6-2.0',
 '6-3.0',
 '19-0.0',
 '21-0.0',
 '21-1.0',
 '21-2.0',
 '21-3.0',
 '31-0.0',
 '34-0.0',
 '36-0.0',
 '36-1.0',
 '36-2.0',
 '36-3.0',
 '37-0.0',
 '37-1.0',
 '37-2.0',
 '37-3.0',
 '38-0.0',
 '38-1.0',
 '38-2.0',
 '38-3.0',
 '39-0.0',
 '39-1.0',
 '39-2.0',
 '39-3.0',
 '40-0.0',
 '40-1.0',
 '40-2.0',
 '40-3.0',
 '41-0.0',
 '41-1.0',
 '41-2.0',
 '41-3.0',
 '42-0.0',
 '42-1.0',
 '42-2.0',
 '42-3.0',
 '43-0.0',
 '43-1.0',
 '43-2.0',
 '43-3.0',
 '44-0.0',
 '44-1.0',
 '44-2.0',
 '44-3.0',
 '45-0.0',
 '45-1.0',
 '45-2.0',
 '45-3.0',
 '46-0.0',
 '46-1.0',
 '46-2.0',
 '46-3.0',
 '47-0.0',
 '47-1.0',
 '47-2.0',
 '47-3.0',
 '48-0.0',
 '48-1.0',
 '48-2.0',
 '48-3.0',
 '49-0.0',
 '49-1.0',
 '49-2.0',
 '49-3.0',
 '50-0.0',
 '50-1.0',
 '50-2.0',
 '50-3.0',
 '51-0.0',
 '51-1.0',
 '51-2.0',
 '51-3.0',
 '52-0.0',
 '53-0.0',
 '53-1.0',
 '53-2.0',
 '53-3.

In [82]:
columns_to_read2 = ['eid', '46-0.0', '47-0.0', '2178-0.0'] 
ukb_outcomes = pd.read_csv('Data/ukb675807.csv',usecols=columns_to_read2)

['eid', '46-0.0', '47-0.0', '2178-0.0']

In [83]:
ukb_outcomes = ukb_outcomes[ukb_outcomes['eid'].astype('int').isin(set3eids)]

In [141]:
ukb_outcomes['max_handgrip'] = ukb_outcomes[['46-0.0', '47-0.0']].max(axis=1)
ukb_outcomes['self_perceived_health'] = ukb_outcomes['2178-0.0']
ukb_outcomes_keep = ukb_outcomes[['eid', 'max_handgrip', 'self_perceived_health']]
ukb_outcomes_keep.loc[:,'self_perceived_health'] = np.where(ukb_outcomes_keep['self_perceived_health'] < 0, np.nan, ukb_outcomes_keep['self_perceived_health'])
ukb_outcomes_keep.loc[:,'self_perceived_health'] = ukb_outcomes_keep['self_perceived_health'].replace({4: 0, 3: 1, 2: 2, 1: 3})

ukb_outcomes_keep.to_csv('Data/HandgripSelfperceived.csv', index=False)

In [142]:
ukb_outcomes_keep.describe()

,eid,max_handgrip,self_perceived_health
count,4.071000e+03,4057.000000,4033.000000
mean,4.550401e+06,32.483362,1.819737
std,1.554289e+06,11.370456,0.747341
min,1.001997e+06,0.000000,0.000000
25%,3.338010e+06,24.000000,1.000000
50%,5.537340e+06,30.000000,2.000000
75%,5.788416e+06,40.000000,2.000000
max,6.024800e+06,86.000000,3.000000


In [108]:
cancer.columns

Index(['eid', 'p40005_i0', 'cancer_inc'], dtype='object')

In [117]:
cancer_all = pd.read_csv('Data/CancerDates.csv')

In [124]:
cancer_all
cancer2 = cancer_all[cancer_all['eid'].isin(set3eids)]
cancer2.shape

(4068, 2)

In [135]:
cancer = pd.read_csv('Data/CancerDates.csv')
cancer['cancer_date'] = pd.to_datetime(cancer['p40005_i0'], errors='coerce')
cancer['cancer_inc'] = (cancer['cancer_date'].notna()).astype(int)
max_date = cancer['cancer_date'].max() + pd.Timedelta(days=1)
cancer['cancer_date'].fillna(max_date, inplace=True)
cancer = cancer[cancer['eid'].isin(set3eids)]
cancer = cancer[['eid', 'cancer_inc', 'cancer_date']]

cancer_time = cancer.merge(basicinfo[['eid', 'date_center.0.0', 'age_center.0.0']], on = 'eid', how = 'inner')
cancer_time['followup'] = cancer_time['cancer_date'] - pd.to_datetime(cancer_time['date_center.0.0'])
cancer_time['cancer_prev'] = (cancer_time['followup'] < pd.Timedelta(0)).astype(int)
cancer_time['cancer_age'] = np.where(
    cancer_time['cancer_prev'] == 0, 
    (cancer_time['followup'].dt.days / 365.25) + cancer_time['age_center.0.0'], 
    pd.NA
)
cancer_time['cancer_inc'] = np.where(
    cancer_time['cancer_prev'] == 1, 
    pd.NA, cancer_time['cancer_inc']
)
cancer_time['cancer_prev'].sum()
cancer_time_save = cancer_time[['eid', 'cancer_prev','cancer_inc', 'cancer_age']]
cancer_time_save.to_csv('Data/FollowUpCancer.csv', index = False)
cancer_time_save

/tmp/ipykernel_2362657/465470314.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  cancer['cancer_date'].fillna(max_date, inplace=True)


,eid,cancer_prev,cancer_inc,cancer_age
0,1747389,0,0,78.355236
1,2805983,0,0,73.80835
2,5599443,1,<NA>,<NA>
3,5602295,0,0,53.555784
4,5733823,0,0,78.943874
...,...,...,...,...
4063,1090274,0,0,67.680356
4064,5746458,0,1,77.381246
4065,1115873,1,<NA>,<NA>
4066,1383092,0,1,72.216975


In [132]:
cancer_time['cancer_date'].isna().sum()

0

In [29]:
_, _, _, _, _, _, cols = get_data({'dset':'allprot', 'target':'mort'})

In [41]:
cols.has_duplicates

False

In [87]:
# icd_pattern = "|".join(["I20", "I21", "I22", "I23", "I24", "I25"])
# icd_columns = [col for col in IHD.columns if col in case_cols]
# IHDtrue = IHD[icd_columns].apply(lambda row: row.str.contains(icd_pattern, na=False), axis=1)
# IHD['col_idx'] = IHDtrue.apply(lambda row: np.where(row.values==True)[0], axis=1)

# import datetime as dt

# def get_earliest_time (row):
#     col_idx = row['col_idx']
#     lowest_date = dt.date.today()
    
#     date_str = "f.41262.0."

#     for idx in col_idx:
#         date = pd.to_datetime(row[date_str + str(idx)])
#         if date.date() < lowest_date:
#             date = lowest_date
#             lowest_date_col = date_str + str(idx)
#     return row[lowest_date_col]

In [82]:
# get_earliest_time(IHD.iloc[0,])

[1]
f.41262.0.1


'2020-02-03'

In [88]:
# IHD.apply(lambda row: get_earliest_time(row), axis =1)

0      2020-02-03
1      2019-12-06
2      2012-08-08
3      2017-11-11
4      2012-03-13
          ...    
362    2002-10-18
363    2014-05-11
364    2013-07-31
365    2019-02-16
366    2007-06-27
Length: 367, dtype: object

In [28]:
# IHD = pd.read_csv('merge_IschemicHeartDisease.date.csv')
# IHD.head()

,f.eid,f.41202.0.0,f.41202.0.1,f.41202.0.2,f.41202.0.3,f.41202.0.4,f.41202.0.5,f.41202.0.6,f.41202.0.7,f.41202.0.8,...,f.41262.0.56,f.41262.0.57,f.41262.0.58,f.41262.0.59,f.41262.0.60,f.41262.0.61,f.41262.0.62,f.41262.0.63,f.41262.0.64,f.41262.0.65
0,1010271,H251,I251,K20,R074,R104,Z538,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1044182,H045,I211,N200,N202,N47,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1113711,I259,J181,J441,K420,R072,R104,R13,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1133974,G459,I214,I633,I634,R074,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1150402,I251,K130,M179,R074,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
